In [5]:
import json
import pandas as pd

file_path = '/Users/pookansmacbookpro/Documents/CEDT Computer Engineering/Sem1_2024/2110403_DataSci/Project/Data_2018-2023/2018/201800029'

import pandas as pd
import json

# Load JSON data (Replace 'file_path.json' with your actual file path)
def extract_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract relevant information from the JSON structure
    records = []

    item = data.get('abstracts-retrieval-response', {}).get('item', {})

    # Extract relevant fields with default values for missing data
    title = item.get('bibrecord', {}).get('head', {}).get('citation-title', '')
    abstract = item.get('bibrecord', {}).get('head', {}).get('abstracts', '')

    authors = []
    for author in data.get('abstracts-retrieval-response', {}).get('authors',{}).get('author',[]):
        name=author.get('preferred-name', {}).get('ce:indexed-name', '')
        if name:
            authors.append(name)

    # Initialize list to hold all classification codes
    subject_areas = []
    subject_area_groups = data.get('abstracts-retrieval-response', {}).get('subject-areas', {})
    if subject_area_groups and isinstance(subject_area_groups.get('subject-area'), list):
        for subject in subject_area_groups['subject-area']:
            if isinstance(subject, dict):
                subject_text = subject.get('@code')
                if subject_text:
                    subject_areas.append(subject_text)

    # Extract date of publication
    publication_date = data.get('abstracts-retrieval-response', {}).get('coredata', {}).get('prism:coverDate', {})

    # Process author affiliations
    affiliations = set()
    author_group = item.get('bibrecord', {}).get('head', {}).get('author-group', {})

    # Check if author_group is a list; if not, wrap it in a list to iterate over it
    if isinstance(author_group, dict):
        author_group = [author_group]  # Convert single dictionary to a list

    for group in author_group:
        affiliation = group.get('affiliation', {})
        affiliation_id = affiliation.get('@afid', '')
        if affiliation_id:
            affiliations.add(affiliation_id)


    # Extract citations/references
    references = []
    tail = item.get('bibrecord', {}).get('tail', None)
    if tail and isinstance(tail, dict):
        bibliography = tail.get('bibliography', {})
        reference = bibliography.get('reference', {})
        if isinstance(reference, dict):  # Check if reference is a dictionary
            ref_title = reference.get('ref-info', {}).get('ref-title', {}).get('ref-titletext', '')
            if ref_title:
                references.append(ref_title)

    # Extract keywords
    keywords = []
    authkeywords = data.get('abstracts-retrieval-response', {}).get('authkeywords')

    if authkeywords and isinstance(authkeywords.get('author-keyword'), list):
        for keyword in authkeywords['author-keyword']:
            if isinstance(keyword, dict):
                keyword_text = keyword.get('$')
                if keyword_text:
                    keywords.append(keyword_text)


    records.append({
    'Title': title if title else None,
    'Abstract': abstract if abstract else None,
    'Author': "; ".join(authors) if authors else None,
    'Subject_Area': "; ".join(subject_areas) if subject_areas else None,
    'Publication_Date': publication_date if publication_date else None,
    'Affiliations_ID': "; ".join(affiliations) if affiliations else None,
    'References': "; ".join(references) if references else None,
    'Keywords': "; ".join(keywords) if keywords else None
})

    return print(records)

extract_data(file_path)

[{'Title': 'Cosmic-Ray Muons Detection by a Spark Chamber', 'Abstract': '© Published under licence by IOP Publishing Ltd. Cosmic ray, originating from several galactic sources, bombards the Earth\'s atmosphere and produces lots of secondary particles. One of them is an unstable charged particle known as "muon" (μ ± ) with speed close to speed of light and lifetime about 2.2 microseconds before decaying into others. In this study, a spark chamber detector with a stack of 10 parallel electrode plates is constructed to detect the cosmic-ray muons which will lose their energies via ionization in the helium filled chamber. When muons passing through the chamber, they left their traces as the ionization paths. Sparks will occur immediately between each stack of parallel electrodes which are connected to a high voltage supply. The muon\'s trajectory can then be visualized by connecting the sparks in each stack. To construct this muon\'s path, two cameras are set to record the events from the 